In [1]:
# This algorithm combines the Fixed Window Counter and Sliding Window Log 
# approaches for a more accurate and efficient solution. Instead of keeping 
# track of every single request’s timestamp as the sliding log does, it focus
# on the number of requests from the last window. So, if you are in 75% of 
# the current window, 25% of the weight would come from the previous window,
# and the rest from the current one:
# In such cases weight will be calculated as:

# weight = (100 - 75)% * lastWindowRequests + currentWindowRequests
# Now, when a new request comes, you add one to that weight (weight + 1).
# If this new total crosses our set limit, we have to reject the request.

# Algorithm:
# i) Keep track of request count for the current and previous window.
# ii) Calculate the weighted sum of requests based on the overlap with the sliding window.
# iii) If the weighted sum is less than the limit, allow the request.

# Note. To calculate the overlap, we can use simple maths.
# Ex. Previous window had 3 req. Current window till now has 4 req and 60% of the 
# current window has elapsed. So we calculate 3*0.4 + 4 = 5 (Can be taken 6 if we are strict)
# And now a req is coming, we basically we will have 6 req in this window. Surely,
# it will only be served if maxReq >= 6. Otherwise it will be dropped.

# Ex. how to calculate how much window has elapsed? Say windowSize = 10 seconds.
# Our windows are like this: 0-9 (Window-0), 10-19 (Window-1), 20-29 (Window-2) etc.
# So, If we are at second = 24, so it is 24/10 = window no. 2. But how much time has passed?
# 24 % 10 = 4, so elapsed = 4/10 = 0.4 i.e., 40%. So, previous window to consider is 60%.

In [8]:
import time as TIME
# We do not need memory for deque here.

class SlidingWindowCounter():
    def __init__(self, windowSize, maxReq):
        self.windowSize = windowSize
        self.maxReq = maxReq
        
        # We need to check current window no and no of req
        # in prev and curr window.
        self.currWindowNo = TIME.time()//windowSize
        self.currWindowReq = 0
        self.prevWindowReq = 0
        
    def handleReq(self, req): # We take req IRL also.
        requestTime = TIME.time() # Current req time.
        # We need to see if this window is same as self.currWindowNo
        windowNo = int(requestTime)//self.windowSize
        timeElapsed = (int(requestTime) % self.windowSize) / self.windowSize # This should not be interger division.
        
        # We can calculate weighted effective no. of requests based on this timeElapsed fraction.
        # But before that we must see if we have entered a new window or not.
        
        if(windowNo != self.currWindowNo):
            # So, we are still in a new window.
            self.currWindowNo = windowNo
            self.prevWindowReq = self.currWindowReq
            self.currWindowReq = 0
            
        # Now, we will calculate the effective no. of requests.
        effectiveReq = (1 - timeElapsed)*self.prevWindowReq + self.currWindowReq
        # Now, we need to see if we can also accomodate our new req.
        
        if(effectiveReq < self.maxReq):
            print("Request can be processed...")
            self.currWindowReq = self.currWindowReq + 1
            return True
        # Otherwise we cannot serve this request.
        print("Request cannot be processed...")
        return False

In [9]:
limiter = SlidingWindowCounter(windowSize = 10, maxReq = 5)

for req in range(10):
    print(f"This is request {req+1}")
    limiter.handleReq(req)
    TIME.sleep(0.4)
    
TIME.sleep(7)
# So, we wait for 7 more seconds to make our window slide.
print(f"This is a new request")
limiter.handleReq(100)

This is request 1
Request can be processed...
This is request 2
Request can be processed...
This is request 3
Request can be processed...
This is request 4
Request can be processed...
This is request 5
Request can be processed...
This is request 6
Request cannot be processed...
This is request 7
Request cannot be processed...
This is request 8
Request cannot be processed...
This is request 9
Request can be processed...
This is request 10
Request cannot be processed...
This is a new request
Request can be processed...


True